# Building  Personalize ReRanking

## Imports 

Python ships with a broad collection of libraries and we need to import those as well as the ones installed to help us like boto3(The AWS SDK) and Pandas/Numpy which are core data science tools

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time


Next you will want to validate that your environment can communicate successfully with Amazon Personalize, the lines below do just that.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [23]:
%store -r dataset_group_arn
%store -r Rerank_arn
%store -r useritems
%store -r bucket

## Create the Solution and Version

In Amazon Personalize a trained model is called a Solution, each Solution can have many specific versions that relate to a given volume of data when the model was trained.

To begin we will list all the recipies that are supported, a recipie is an algorithm that has not been trained on your data yet. After listing you'll select one and use that to build your model.

### Select Recipe

In [4]:
recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking" 
# aws-personalized-ranking selected for this solution. 

### Create and Wait for Solution

First you will create the solution with the API, then you will create a version. It will take several minutes to train the model and thus create your version of a solution. Once it gets started and you are seeing the in progress notifications it is a good time to take a break, grab a coffee, etc.

#### Create Solution

In [5]:
create_solution_response = personalize.create_solution(
    name = "personalize-demo-soln-personal-ranking",   # Please change the solution name if you are changing the recipe
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

prr_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:962222257213:solution/personalize-demo-soln-personal-ranking",
  "ResponseMetadata": {
    "RequestId": "4122d70a-db86-4d11-8577-53aa3511ae82",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 25 Feb 2020 05:09:41 GMT",
      "x-amzn-requestid": "4122d70a-db86-4d11-8577-53aa3511ae82",
      "content-length": "108",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create Solution Version

In [6]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = prr_solution_arn
)

prr_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:962222257213:solution/personalize-demo-soln-personal-ranking/c3b11a9a",
  "ResponseMetadata": {
    "RequestId": "e0e61f56-1d22-4179-a930-0c84d42ea0be",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 25 Feb 2020 05:09:41 GMT",
      "x-amzn-requestid": "e0e61f56-1d22-4179-a930-0c84d42ea0be",
      "content-length": "124",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Solution Version to Have ACTIVE Status

This will take at least 20 minutes.

In [14]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = prr_solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: ACTIVE


#### Get Metrics of Solution Version

Now that your solution and version exists, you can obtain the metrics for it to judge its performance. These metrics are not particularly good as it is a demo set of data, but with larger more compelx datasets you should see improvements.

In [15]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = prr_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:962222257213:solution/personalize-demo-soln-personal-ranking/c3b11a9a",
  "metrics": {
    "coverage": 0.0014,
    "mean_reciprocal_rank_at_25": 0.0034,
    "normalized_discounted_cumulative_gain_at_10": 0.0051,
    "normalized_discounted_cumulative_gain_at_25": 0.0067,
    "normalized_discounted_cumulative_gain_at_5": 0.004,
    "precision_at_10": 0.0009,
    "precision_at_25": 0.0006,
    "precision_at_5": 0.0012
  },
  "ResponseMetadata": {
    "RequestId": "5b207d94-3a6d-44f3-898f-ab22097aec94",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 25 Feb 2020 06:43:12 GMT",
      "x-amzn-requestid": "5b207d94-3a6d-44f3-898f-ab22097aec94",
      "content-length": "421",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Create and Wait for the Campaign

Now that you have a working solution version you will need to create a campaign to use it with your applications. A campaign is simply a hosted copy of your model. Again there will be a short wait so after executing you can take a quick break while the infrastructure is being provisioned.

#### Create Campaign

In [16]:
create_campaign_response = personalize.create_campaign(
    name = "personalize-campaign-reranking",
    solutionVersionArn = prr_solution_version_arn,
    minProvisionedTPS = 1
)

prr_campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

ResourceAlreadyExistsException: An error occurred (ResourceAlreadyExistsException) when calling the CreateCampaign operation: Another resource with Arn arn:aws:personalize:us-east-1:962222257213:campaign/personalize-campaign-reranking already exists.

#### Wait for Campaign to Have ACTIVE Status

In [17]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = prr_campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: ACTIVE


## Get Sample ReRanking



In [18]:
# Getting random itemlist
datasample = useritems.sample(n=20)
sampleitemlist = []
for index, row in datasample.iterrows():
    sampleitemlist.append(row[1])
    
    
#Getting a random user
user_id, item_id, _ = useritems.sample().values[0]

# print("USER: {}".format(item_id))
# print("ItemList:")
# print(sampleitemlist)

In [20]:
# First load items into memory
%store -r bucket
allitemuri = f's3://{bucket}/items_w_metadata.csv'
items = pd.read_csv(allitemuri, sep=',', usecols=[0,1], names=['asin', 'title'], index_col='asin',header=0)

# print(items)

def get_allstore_products(asin):
    """
    Takes in an ID, returns a title
    """
    asin = str(asin)
    return items.loc[asin]['title']

#### Call GetRecommendations

Using the user that you obtained above, the lines below will get recommendations for you and return the list of movies that are recommended.


In [21]:
get_recommendations_response = personalize_runtime.get_personalized_ranking(
    campaignArn=prr_campaign_arn,
    inputList=sampleitemlist,
    userId = str(user_id)
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['personalizedRanking']


recommendation_list = []

for item in item_list:
    title = get_allstore_products(item['itemId'])
    recommendation_list.append(title)  
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['Item Description'])
recommendations_df

Recommendations for user:  A1LWN1D2XNSSYJ


,Item Description
0,Butler Creek Lula Universal Pistol Magazine Lo...
1,Pro-Shot .22-.45 Caliber Brass Patch Holder
2,HiMart Side Picatinny Mount for Tactical Flash...
3,Harris Engineering S-BRM Hinged Base 6 - 9-Inc...
4,NaN
5,"Magpul iPhone 4 Executive Field Case, Black"
6,UTG 2-in-1 Tactical LED Flashlight with Red Laser
7,8&quot; Round Birchwood Casey Shoot - N - C Se...
8,Kershaw 1660 Ken Onion Leek Folding Knife with...
9,308 SR-25 / AR-10 Lower Receiver Vise Block Ar...


## Update Lambda Configuration

In [24]:
lambdaclient = boto3.client('lambda')

response = lambdaclient.get_function(
    FunctionName=Rerank_arn,
)

Environment = response['Configuration']['Environment']
print(Environment)

#update Env var

Environment['Variables']['Campaign_ARN'] = prr_campaign_arn

response = lambdaclient.update_function_configuration(
    FunctionName=Rerank_arn,
    Environment=Environment
)

{'Variables': {'DDB_TABLE': 'teststr-Items', 'SEARCH_ARN': 'arn:aws:lambda:us-east-1:962222257213:function:teststr-Search', 'RANKING_ARN': 'arn:aws:personalize:us-east-1:387269085412:campaign/personalize-demo-soln-ranking', 'ESENDPOINT': 'vpc-teststr-domain-yyb6n76pkinseg5whgedynqqve.us-east-1.es.amazonaws.com'}}


## Review

Using the codes above you have successfully trained a deep learning model to generate Allstore Merchandise recommendations based on prior user behavior. Think about other types of problems where this data is available and what it might look like to build a system like this to offer those recommendations.

Now you are ready to move onto the next notebook `3.Building_Campaign_P-rank.ipynb`

